## transferlearning002.ipynb

#### transferlearning001에서는 원래 데이터 셋에서 10%만의 데이터로 학습을 시키더라도
#### 우리가 직접 만들어본 모델에 원래 데이터 셋을 학습시킨 것보다 더좋은 성능을 가진다는 것을 transferlearning(전이학습)을 통해서 알게 되었습니다.
#### feature extraction이 잘 되었다.

#### 이번 transfer learning 에서는 fine-tuning 이라는 기법을 알아보려고 합니다.

In [2]:
!wget -q https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip


In [16]:
from helper_functions import unzip_data, walk_through_dir, create_tensorboard_callback

In [2]:
unzip_data("10_food_classes_10_percent.zip")

In [3]:
walk_through_dir("10_food_classes_10_percent")

10_food_classes_10_percent 폴더에는 2개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent/test 폴더에는 10개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent/test/ice_cream 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/chicken_curry 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/steak 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/sushi 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/chicken_wings 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/grilled_salmon 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/hamburger 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/pizza 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/ramen 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/test/fried_rice 폴더에는 0개의 폴더와 250개의 파일이 존재합니다.
10_food_classes_10_percent/train 폴더에는 10개의 폴더와 0개의 파일이 존재합니다.
10_food_classes_10_percent/train/ice_cream 폴더에는 0개의 폴더와 75개의 파일이 존재합니다.
10_food_classes_10_percent/train

In [4]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"


In [17]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# transferlearning_001 에서는 지금까지 ImageDataGenerator를 사용했습니다. 
# 잘 동작도 하고 많이 사용하는 방법이다.
# transferlearning_002에서는 image_data_from_directory라는 함수를 사용한다.

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size = IMAGE_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE
)

test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir,
    image_size = IMAGE_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE
)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


### ImageDataGenerator : python generator
### tf.keras.preprocessing.image_dataset_from_directory : tf.data.Dataset 오브젝트
### tf.data.Dataset API는 큰 데이터 셋에 대해서 좀 더 효율적이다. 그래서 많이 사용함
### tf.data.Dataset API는 관련된 메서드들이 많이 존재한다.

In [8]:
train_data_10_percent

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

#### (None, 224, 224, 3) : None은 배치사이즈, 224는 높이, 224는 너비, 3은 컬러 채널(RGB)인 이미지의 텐서 shape(모양)
#### (None, 10) : None은 배치사이즈, 10은 가능한 라벨의 갯수 (우리가 실습하는 데이터 셋은 10개의 종류가 있는 것을 자동으로 판단)인 텐서 shape
#### 이 텐서들은 모두 tf.float32 자료형을 가지고 있다.

#### batch_size가 None인 이유 : 학습하는 동안에만 사용하기 때문이다.
#### None의 의미는 placeholder(자리를 차지하는)

In [9]:
# class_names() 메서드 : 클래스 목록화

train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [10]:
# take() 메서드 : 데이터 배치의 사례를 확인
for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[8.36989822e+01 8.36989822e+01 8.16989822e+01]
   [8.69030609e+01 8.69030609e+01 8.49030609e+01]
   [8.62193909e+01 8.62193909e+01 8.42193909e+01]
   ...
   [1.06571404e+02 1.02571404e+02 1.01571404e+02]
   [1.06357140e+02 1.02357140e+02 1.01357140e+02]
   [1.06714287e+02 1.02714287e+02 1.01714287e+02]]

  [[8.05000000e+01 8.05000000e+01 7.85000000e+01]
   [8.46428604e+01 8.46428604e+01 8.26428604e+01]
   [8.78724518e+01 8.78724518e+01 8.58724518e+01]
   ...
   [1.07928574e+02 1.03928574e+02 1.02928574e+02]
   [1.07933678e+02 1.03933678e+02 1.02933678e+02]
   [1.08954086e+02 1.04954086e+02 1.03954086e+02]]

  [[7.76326523e+01 7.76326523e+01 7.56326523e+01]
   [8.02857208e+01 8.02857208e+01 7.82857208e+01]
   [8.44744873e+01 8.44744873e+01 8.24744873e+01]
   ...
   [1.06214287e+02 1.02214287e+02 1.01214287e+02]
   [1.05500015e+02 1.01500015e+02 1.00500015e+02]
   [1.06785751e+02 1.02785751e+02 1.01785751e+02]]

  ...

  [[1.39000000e+02 1.60000000e+02 1.79000000e+02]
   [1

In [11]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [18]:
# Model 1
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

base_model.trainable = False 

input = tf.keras.layers.Input(
    shape=(224, 224, 3),
    name="input_layer"
)

x = base_model(input)
print(f"base_model의 shape : {x.shape}")

x = tf.keras.layers.GlobalAveragePooling2D(
    name="global_average_pooling_layer"
)(x)
print(f"global_average_pooling_layer 이후의 shape {x.shape}")

outputs = tf.keras.layers.Dense(
    10, 
    activation="softmax",
    name="output_layer"
)(x)

model_0 = tf.keras.Model(input, outputs)
model_0.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

history_10_percent = model_0.fit(
    train_data_10_percent,
    epoch=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data = test_data_10_percent,
    validation_steps = len(test_data_10_percent),
    callbacks = [create_tensorboard_callback(
        "transfer_learning", "10_percent_feature_extract"
    )]
)

base_model의 shape : (None, 7, 7, 1280)
global_average_pooling_layer 이후의 shape (None, 1280)


NameError: name 'tf' is not defined